## Using PyTorch Lightning for a regression task with the dataset Boston Housing Prices (BHPD) Dataset

*Remark : This code was adapted from the Boston housing example from Fidle online courses (Jean-Luc Parouty)*

This notebook consists in predicting **housing prices** from a set of house features. It allows discovering **PyTorch Lightning** with simple fully connected neural networks for a regression task.


The **[Boston Housing Prices Dataset](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html)** consists in estimating the price of houses in different neighbourhoods of Boston.  

The following variables will be used to predict the house prices:

 - CRIM: This is the per capita crime rate by town
 - ZN: This is the proportion of residential land zoned for lots larger than 25,000 sq.ft
 - INDUS: This is the proportion of non-retail business acres per town
 - CHAS: This is the Charles River dummy variable (this is equal to 1 if tract bounds river; 0 otherwise)
 - NOX: This is the nitric oxides concentration (parts per 10 million)
 - RM: This is the average number of rooms per dwelling
 - AGE: This is the proportion of owner-occupied units built prior to 1940
 - DIS: This is the weighted distances to five Boston employment centers
 - RAD: This is the index of accessibility to radial highways
 - TAX: This is the full-value property-tax rate per 10,000 dollars
 - PTRATIO: This is the pupil-teacher ratio by town
 - B: This is calculated as 1000(Bk — 0.63)^2, where Bk is the proportion of people of African American descent by town
 - LSTAT: This is the percentage lower status of the population
 - MEDV: This is the median value of owner-occupied homes in 1000 dollars



 ### Step 1 - Import and initialize the data

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

import lightning as L

In [37]:
dataset = pd.read_csv("./boston_housing_scikit_learn_3.csv")

x_load = np.array([dataset[name] for name in ["CRIM","ZN","INDUS","CHAS","NOX","RM","AGE","DIS","RAD","TAX","PTRATIO","B","LSTAT","MEDV"]])
y_load = np.array(dataset["MEDV"])

x_train, y_train = x_load[:,:325].transpose(), y_load[:325]
x_test, y_test = x_load[:,325:].transpose(), y_load[325:]

x_train, y_train = torch.tensor(x_train).float(), torch.tensor(y_train).float()
x_test, y_test = torch.tensor(x_test).float(), torch.tensor(y_test).float()

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

torch.Size([325, 14])
torch.Size([181, 14])
torch.Size([325])
torch.Size([181])


In [38]:
#show one raw observation

print('input 0:', x_train[0,:])
print('output 0:', y_train[0])


input 0: tensor([6.3200e-03, 1.8000e+01, 2.3100e+00, 0.0000e+00, 5.3800e-01, 6.5750e+00,
        6.5200e+01, 4.0900e+00, 1.0000e+00, 2.9600e+02, 1.5300e+01, 3.9690e+02,
        4.9800e+00, 2.4000e+01])
output 0: tensor(24.)


In [39]:
#data normalization

mean = x_train.mean()
std  = x_train.std()
x_train = (x_train - mean) / std
x_test  = (x_test  - mean) / std

## Step 2 - Instanciate and train a model
Information about:
 - [Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers)
 - [Activation](https://www.tensorflow.org/api_docs/python/tf/keras/activations)
 - [Loss](https://www.tensorflow.org/api_docs/python/tf/keras/losses)
 - [Metrics](https://www.tensorflow.org/api_docs/python/tf/keras/metrics)


In [40]:
class model_v1(L.LightningModule):
    """
    Basic fully connected neural-network for tabular data
    """
    def __init__(self, num_vars):
        super().__init__()
        self.num_vars = num_vars
        self.training_step_outputs = []
        self.layers = nn.Sequential(
                nn.Linear(self.num_vars, 64),
                nn.ReLU(),
                nn.Linear(64, 64),
                nn.ReLU(),
                nn.Linear(64, 1)
            )

    def forward(self, x):
        return self.layers(x)

    def backward(self, loss):
        self.training_step_outputs.append(loss.detach().cpu().numpy())
        loss.backward()

    def configure_optimizers(self):
        otimizer = torch.optim.Adam(self.parameters(),lr=1e-3)
        return otimizer

    def training_step(self, batch):
        # training_step defines the train loop.
        x, y = batch
        x = x.view(-1,self.num_vars)   #flatten the observation before using fully-connected layers
        z = self.encoder(x)
        loss = F.mse_loss(z, y)
        return loss

In [41]:
#instanciate the model
model = model_v1( x_train[0,:].shape[0])
trainer = L.Trainer(min_epochs=10, max_epochs=60)
trainer.fit(model, (x_train, y_train))

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name    | Type       | Params | Mode 
-----------------------------------------------
0 | encoder | Sequential | 5.2 K  | train
-----------------------------------------------
5.2 K     Trainable params
0         Non-trainable params
5.2 K     Total params
0.021     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name    | Type       | Params | Mode 
-----------------------------------------------
0 | encoder | Sequential | 5.2 K  |

Training: |          | 0/? [00:00<?, ?it/s]

<ipython-input-40-26bf6b813c1f>:34: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(z, y)
INFO: `Trainer.fit` stopped: `max_epochs=60` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=60` reached.


## Step 3 - test the model



In [1]:
print(model.training_step_outputs)
plt.plot(model.training_step_outputs)
plt.title('Loss')

NameError: name 'model' is not defined

In [43]:
Y_train_pred=model(x_train[:,:].float())
Y_test_pred=model(x_test[:,:].float())

mse_train = [torch.mean((Y_train_pred-y_train)*(Y_train_pred-y_train)).detach().numpy()]
mse_test = [torch.mean((Y_test_pred-y_test)*(Y_test_pred-y_test)).detach().numpy()]

print(f"The mean of the last loss for the train is: {mse_train[0]}.")
print(f"The mean of the last loss for the test is: {mse_test[0]}.")

The mean of the last loss for the train is: 153.2789306640625.
The mean of the last loss for the test is: 125.24944305419922.


In [57]:
my_data_centered_reduced = x_test[0,:]
my_data = torch.tensor(my_data_centered_reduced)

predictions = model(my_data).detach().numpy()[0]

print(f"Prediction : {predictions} K$")

Prediction : 24.790102005004883 K$


<ipython-input-57-cc7f0912deef>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  my_data = torch.tensor(my_data_centered_reduced)
